# Sensor Projection

In this tutorial, we will demonstrate how to define a **sensor geometry** and project its trace onto the **Earth**.

## Setup

Let's import the necessary dependencies:

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.mathematics.geometry.d2 import Object as Object2d
from ostk.mathematics.geometry.d2.objects import Point as Point2d
from ostk.mathematics.geometry.d2.objects import Polygon as Polygon2d
from ostk.mathematics.geometry.d3.objects import Point as Point3d
from ostk.mathematics.geometry.d3.objects import Polygon as Polygon3d
from ostk.mathematics.geometry.d3.objects import Ellipsoid
from ostk.mathematics.geometry.d3.objects import Pyramid

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.object import Geometry
from ostk.physics.environment.objects.celestial_bodies import Earth

In [ ]:
pip freeze | grep math

---

## Computation

### Scene

We first set up a simple scene, with the Earth only (the default `Environment` will suffice here):

In [ ]:
environment = Environment.default() ;

Then, we access the `Earth` object (managed by the `Environment`):

In [ ]:
earth = environment.access_object_with_name("Earth")

Once the `Earth` has been obtained, we can also get its geometry (an `Ellipsoid`, defined in `ITRF` in this case):

In [ ]:
earth_geometry = earth.get_geometry_in(Frame.ITRF())

### Sensor

Let's define a pyramidal geometry:

In [ ]:
apex = Point3d(7000e3, 0.0, 0.0)
base = Polygon3d(Polygon2d([Point2d(-1.0, -1.0), Point2d(+1.0, -1.0), Point2d(+1.0, +1.0), Point2d(-1.0, +1.0)]), apex - np.array((0.8, 0.0, 0.0)), np.array((0.0, 1.0, 0.0)), np.array((0.0, 0.0, 1.0)))

pyramid = Pyramid(base, apex)

That we express in `ITRF`:

In [ ]:
sensor_geometry = Geometry(pyramid, Frame.ITRF())

### Intersection

Now that we have both Earth and sensor geometries clearly defined, we can compute their intersection:

In [ ]:
intersection_ITRF = sensor_geometry.intersection_with(earth_geometry)

And convert this 3D intersection into a 2D set of geodetic points:

In [ ]:

# intersection_points.append( = [Point2d(lla.get_longitude().in_degrees(), lla.get_latitude().in_degrees()) for lla in [LLA.cartesian(point_ITRF.as_vector(), Earth.equatorial_radius, Earth.flattening) for point_ITRF in intersection_ITRF.access_composite().access_object_at(0).as_line_string()]]


intersection_points = [Point2d(lla.get_longitude().in_degrees(), lla.get_latitude().in_degrees()) for lla in [LLA.cartesian(point_ITRF.as_vector(), Earth.equatorial_radius, Earth.flattening) for point_ITRF in intersection_ITRF.access_composite().access_object_at(0).as_line_string()]]

We further convert this set into a Pandas Dataframe, as a very convenient way for storing / managing data in Python:

In [ ]:
intersection_df = pd.DataFrame([[float(intersection_point.x()), float(intersection_point.y())] for intersection_point in intersection_points], columns=['Longitude', 'Latitude']) ;

---

## Visualization

Table:

In [ ]:
intersection_df.head()

Now, we're ready to visualize the intersection on a map!

In [ ]:
figure = go.Figure(
    data = go.Scattergeo(
        lon = intersection_df['Longitude'],
        lat = intersection_df['Latitude'],
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'red',
        )
    ),
    layout = go.Layout(
        title = None,
        showlegend = False, 
        geo = dict(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
)

figure.show()

It is also possible to obtain the `WKT` representation of the intersection polygon:

In [ ]:
Polygon2d(intersection_points).to_string(Object2d.Format.WKT)

---